In [1]:
import jieba
import re

In [2]:
def text_segment(text):
    tmp_list = re.split('[?!。？！]', text)
    if tmp_list[-1] == '':
        tmp_list = tmp_list[:-1]
    return tmp_list

In [3]:
def build_labels(seg_context, answer_start):
    labels = []
    all_len = 0
    for item in seg_context:
        if all_len <= int(answer_start) <= len(item) + all_len:
            labels.append(str(1))
        else:
            labels.append(str(0))
        all_len += len(item) + 1
    return labels[:len(seg_context)]

In [4]:
def data_builder(origin):
    new = []
    for idx in range(len(origin['data'])):
        item = origin['data'][idx]
        paragraphs = item['paragraphs']
        for para in paragraphs:
            qas_list = para['qas']
            context = para['context']
            context = context.replace('\n', '  ') # 遇害人的手提电话被凶手抢去。而该手提电机在案发后曾经被人\n使用，因此能够凭借通话纪录，
            seg_context = text_segment(context)
            all_labels = [0] * len(seg_context)
            all_answers = []
            for qas in qas_list:
                labels = build_labels(seg_context, str(qas['answers'][0]['answer_start']).strip('。'))
                tmp = {'select_sentence': seg_context[labels.index(max(labels))], 'question':str(qas['question']), 'answer':str(qas['answers'][0]['text']).strip('。'), 'answer_start':str(qas['answers'][0]['answer_start']), 'char_at_answer_start': context[qas['answers'][0]['answer_start']]}
                all_answers.append(tmp)
                pa_list.append(str(" ".join(list(seg_context[labels.index(max(labels))]))) + " [SEP] " + str(" ".join(list(qas['answers'][0]['text']))))
                q_list.append(" ".join(str(qas['question'])))
                count = 0
                for x, y in zip (all_labels, labels):
                    all_labels[count] = str(int(x) + int(y))
                    count += 1
            new.append(json.dumps({"doc":str('\n'.join(seg_context)), "labels":str('\n'.join(all_labels)), "summaries":str(all_answers)}, ensure_ascii=False))
    return new

In [5]:
all_line = ''
pa_list = []
q_list = []
#with open('../../../../resources/corpus/mrc/cmrc2018/data/cmrc2018_train.json', mode='r', encoding='utf-8') as f:
with open('../../../../resources/corpus/mrc/cmrc2018/squad-style-data/cmrc2018_train.json', mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    all_line = ''.join(lines)

import json 
train_data = json.loads(all_line)
print(data_builder(train_data)[:1])


['{"doc": "范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机\\n1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世\\n范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业\\n范廷颂于1940年在河内大修道院完成神学学业\\n范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务\\n1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民\\n1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内\\n翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕\\n1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」\\n由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作\\n范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等\\n1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺\\n1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机\\n范廷颂在1995年至2001年期间出任天主教越南主教团主席\\n2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务\\n范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行", "labels": "0\\n2\\n1\\n0\\n0\\n0\\n0\\n0\\n0\\n0\\n0\\n0\\n1\\n0\\n0\\n1", "summaries": "[{\'select_sentence\': \'1963年被任为主教；1990年被擢升为天主教河内总教区

In [6]:
print(len(pa_list))

10142


In [7]:
with open('train_q_list.txt', mode='w', encoding='utf-8') as f:
    for line in q_list:
        f.write(line.strip() + '\n')
with open('train_pa_list.txt', mode='w', encoding='utf-8') as f:
    for line in pa_list:
        f.write(line.strip() + '\n')

In [8]:
all_line = ''
pa_list = []
q_list = []
with open('../../../../resources/corpus/mrc/cmrc2018/squad-style-data/cmrc2018_trial.json', mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    all_line = ''.join(lines)

import json 
test_data = json.loads(all_line)
print(data_builder(test_data)[:1])
print(len(pa_list))

['{"doc": "基于《跑跑卡丁车》与《泡泡堂》上所开发的游戏，由韩国Nexon开发与发行\\n中国大陆由盛大游戏运营，这是Nexon时隔6年再次授予盛大网络其游戏运营权\\n台湾由游戏橘子运营\\n玩家以水枪、小枪、锤子或是水炸弹泡封敌人(玩家或NPC)，即为一泡封，将水泡击破为一踢爆\\n若水泡未在时间内踢爆，则会从水泡中释放或被队友救援(即为一救援)\\n每次泡封会减少生命数，生命数耗完即算为踢爆\\n重生者在一定时间内为无敌状态，以踢爆数计分较多者获胜，规则因模式而有差异\\n以2V2、4V4随机配对的方式，玩家可依胜场数爬牌位(依序为原石、铜牌、银牌、金牌、白金、钻石、大师) ，可选择经典、热血、狙击等模式进行游戏\\n若游戏中离，则4分钟内不得进行配对(每次中离+4分钟)\\n开放时间为暑假或寒假期间内不定期开放，8人经典模式随机配对，采计分方式，活动时间内分数越多，终了时可依该名次获得奖励", "labels": "0\\n0\\n0\\n2\\n0\\n0\\n0\\n1\\n1\\n0", "summaries": "[{\'select_sentence\': \'玩家以水枪、小枪、锤子或是水炸弹泡封敌人(玩家或NPC)，即为一泡封，将水泡击破为一踢爆\', \'question\': \'生命数耗完即算为什么？\', \'answer\': \'踢爆\', \'answer_start\': \'127\', \'char_at_answer_start\': \'踢\'}, {\'select_sentence\': \'若游戏中离，则4分钟内不得进行配对(每次中离+4分钟)\', \'question\': \'若游戏中离，则多少分钟内不得进行配对？\', \'answer\': \'4分钟\', \'answer_start\': \'301\', \'char_at_answer_start\': \'4\'}, {\'select_sentence\': \'玩家以水枪、小枪、锤子或是水炸弹泡封敌人(玩家或NPC)，即为一泡封，将水泡击破为一踢爆\', \'question\': \'玩家用什么泡封敌人？\', \'answer\': \'玩家以水枪、小枪、锤子或是水炸弹泡封敌人\', \'answer_start\': \'85

In [9]:
with open('test_q_list.txt', mode='w', encoding='utf-8') as f:
    for line in q_list:
        f.write(line.strip() + '\n')
with open('test_pa_list.txt', mode='w', encoding='utf-8') as f:
    for line in pa_list:
        f.write(line.strip() + '\n')